# About dataset
This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database.

In particular, all patients here are females at least 21 years old of Pima Indian heritage.

# Importing Essential Libraries 

In [ ]:
import shap 
import warnings  
warnings.filterwarnings('ignore')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
from pdpbox import pdp, get_dataset, info_plots
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.tree import DecisionTreeClassifier
import os
import seaborn as sns
print(os.listdir("../input"))
from sklearn.metrics import classification_report
import itertools
plt.style.use('fivethirtyeight')

In [ ]:
df=pd.read_csv('../input/diabetes.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

# Data Cleaning & Data Preparation
In this step we will find missing entries, if there then fill them with median or mean values, checking data types of all the features to find any inconsistency.

In [ ]:
df.isna().any() # No NAs

In [ ]:
print(df.dtypes)

In [ ]:
print(df.info())

In [ ]:
df.head(10)

It seems from the above table that there are zero entries in BMI, Blood Pressure,Glucose, Skin Thickness and Insulin which are meaningless so we will fill it with their median values before fitting it into the machine learning models.

Replacing zero entries BMI, Blood Pressure,Glucose, Skin Thickness and Insulin with their median values

In [ ]:
# Calculate the median value for BMI
median_bmi = df['BMI'].median()
# Substitute it in the BMI column of the
# dataset where values are 0
df['BMI'] = df['BMI'].replace(
    to_replace=0, value=median_bmi)

median_bloodp = df['BloodPressure'].median()
# Substitute it in the BloodP column of the
# dataset where values are 0
df['BloodPressure'] = df['BloodPressure'].replace(
    to_replace=0, value=median_bloodp)

# Calculate the median value for PlGlcConc
median_plglcconc = df['Glucose'].median()
# Substitute it in the PlGlcConc column of the
# dataset where values are 0
df['Glucose'] = df['Glucose'].replace(
    to_replace=0, value=median_plglcconc)

# Calculate the median value for SkinThick
median_skinthick = df['SkinThickness'].median()
# Substitute it in the SkinThick column of the
# dataset where values are 0
df['SkinThickness'] = df['SkinThickness'].replace(
    to_replace=0, value=median_skinthick)

# Calculate the median value for SkinThick
median_skinthick = df['Insulin'].median()
# Substitute it in the SkinThick column of the
# dataset where values are 0
df['Insulin'] = df['Insulin'].replace(
    to_replace=0, value=median_skinthick)

# Data Visualization

In [ ]:
df['Outcome'].value_counts().plot.bar();

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
df['Outcome'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('target')
ax[0].set_ylabel('')
sns.countplot('Outcome',data=df,ax=ax[1])
ax[1].set_title('Outcome')
plt.show()

# Histogram
In this step we will analyze histogram of all the independent variables

In [ ]:
columns=df.columns[:8]
plt.subplots(figsize=(18,15))
length=len(columns)
for i,j in itertools.zip_longest(columns,range(length)):
    plt.subplot((length/2),3,j+1)
    plt.subplots_adjust(wspace=0.2,hspace=0.5)
    df[i].hist(bins=20,edgecolor='black')
    plt.title(i)
plt.show()

# Histogram of Diabetic Cases

In [ ]:
df1=df[df['Outcome']==1]
columns=df.columns[:8]
plt.subplots(figsize=(18,15))
length=len(columns)
for i,j in itertools.zip_longest(columns,range(length)):
    plt.subplot((length/2),3,j+1)
    plt.subplots_adjust(wspace=0.2,hspace=0.5)
    df1[i].hist(bins=20,edgecolor='black')
    plt.title(i)
plt.show()

In [ ]:

sns.pairplot(df, hue = 'Outcome', vars = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI','DiabetesPedigreeFunction','Age'] )

In [ ]:
sns.jointplot("Pregnancies", "Insulin", data=df, kind="reg")

In [ ]:
sns.jointplot("BloodPressure", "Insulin", data=df, kind="reg")

# Feature Engineering
Now, its time to add important features to the dataset discover some effective features before fitting it into machine learning models.

**Feature 1 : BMI Descriptor**
I m adding BMI Descriptor feature as we know : If you have a BMI of:

- Under 18.5 – you are considered underweight and possibly malnourished.
- 18.5 to 24.9 – you are within a healthy weight range for young and middle-aged adults.
- 25.0 to 29.9 – you are considered overweight.
- Over 30 – you are considered obese.

In [ ]:
def set_bmi(row):
    if row["BMI"] < 18.5:
        return "Under"
    elif row["BMI"] >= 18.5 and row["BMI"] <= 24.9:
        return "Healthy"
    elif row["BMI"] >= 25 and row["BMI"] <= 29.9:
        return "Over"
    elif row["BMI"] >= 30:
        return "Obese"

In [ ]:
df = df.assign(BM_DESC=df.apply(set_bmi, axis=1))

df.head()

**Feature 2:** Insulin Indicative Range 
If insulin level (2-Hour serum insulin (mu U/ml)) is >= 16 and <= 166, then it is **normal** range else it is considered as **Abnormal**

In [ ]:
def set_insulin(row):
    if row["Insulin"] >= 16 and row["Insulin"] <= 166:
        return "Normal"
    else:
        return "Abnormal"

In [ ]:
df = df.assign(INSULIN_DESC=df.apply(set_insulin, axis=1))

df.head()

In [ ]:
sns.countplot(data=df, x = 'INSULIN_DESC', label='Count')

AB, NB = df['INSULIN_DESC'].value_counts()
print('Number of patients Having Abnormal Insulin Levels: ',AB)
print('Number of patients Having Normal Insulin Levels: ',NB)

It seems from the above plot that more than 500 patients have Abnormal Insulin Levels where as around 250 patients have Normal Insulin Levels.

In [ ]:
sns.countplot(data=df, x = 'BM_DESC', label='Count')

UD,H,OV,OB = df['BM_DESC'].value_counts()
print('Number of patients Having Underweight BMI Index: ',UD)
print('Number of patients Having Healthy BMI Index: ',H)
print('Number of patients Having Overweigth BMI Index: ',OV)
print('Number of patients Having Obese BMI Index: ',OB)

In [ ]:
g = sns.FacetGrid(df, col="INSULIN_DESC", row="Outcome", margin_titles=True)
g.map(plt.scatter,"Glucose", "BloodPressure",  edgecolor="w")
plt.subplots_adjust(top=1.1)

In [ ]:
g = sns.FacetGrid(df, col="INSULIN_DESC", row="Outcome", margin_titles=True)
g.map(plt.scatter,"DiabetesPedigreeFunction", "BloodPressure",  edgecolor="w")
plt.subplots_adjust(top=1.1)

In [ ]:
g = sns.FacetGrid(df, col="Outcome", row="INSULIN_DESC", margin_titles=True)
g.map(plt.hist, "Age", color="red")
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Disease by INSULIN and Age');

In [ ]:
sns.boxplot(x="Age", y="INSULIN_DESC", hue="Outcome", data=df);

It seems from the above plot that patients having normal insulin levels are more diabetic within the age range from 25 and 42 where as patients having anormal insulin levels are more diabetic in the age range of late 20's to mid 40's.

In [ ]:
sns.boxplot(x="Age", y="BM_DESC", hue="Outcome", data=df);

From the above plot it is evident that patients who are obese as per BMI index are more diabetic in early age of 25 where as patients who are overweight are prone to diabetes in early 30's

# Plotting Correlation Plot (Heat Map)

**Pearson’s correlation coefficient** is the test statistics that measures the statistical relationship, or association, between two continuous variables.  It is known as the best method of measuring the association between variables of interest because it is based on the method of covariance.  It gives information about the magnitude of the association, or correlation, as well as the direction of the relationship.

The value of Pearson's Correlation Coefficient can be between -1 to +1. 1 means that they are highly correlated and 0 means no correlation.

A heat map is a two-dimensional representation of information with the help of colors. Heat maps can help the user visualize simple or complex information.

In [ ]:
df1=df.drop(['Outcome'],axis=1)

In [ ]:
plt.figure(figsize=(12,10))  # on this line I just set the size of figure to 12 by 10.
p=sns.heatmap(df1.corr(), annot=True,cmap ='RdYlGn')

# Segregating Feature & Target Variable

I have taken X as Feature variable and y as target variable.

In [ ]:
X=pd.get_dummies(df)
cols_drop=['Outcome','BM_DESC_Under']
X=X.drop(cols_drop,axis=1)

y = df['Outcome']

In [ ]:
X.head()

In [ ]:
y.head()

# Train Test Split
The data we use is usually split into training data and test data. The training set contains a known output and the model learns on this data in order to be generalized to other data later on. We have the test dataset (or subset) in order to test our model’s prediction on this subset.

![zscore](https://i.ibb.co/7G2RMgX/TRAIN.png)

**Stratify property in train test split**
This stratify parameter makes a split so that the proportion of values in the sample produced will be the same as the proportion of values provided to parameter stratify.
For example, if variable y is a binary categorical variable with values 0 and 1 and there are 25% of zeros and 75% of ones, stratify=y will make sure that your random split has 25% of 0's and 75% of 1's.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,stratify=y, random_state = 1234)

# Data Normalization
I have used Z-score normalization.
Z-scores are linearly transformed data values having a mean of zero and a standard deviation of 1.
Z-scores are also known as standardized scores; they are scores (or data values) that have been given a common standard.

If the population mean and population standard deviation are known, the standard score of a raw score x[1] is calculated as

![zscore](https://i.ibb.co/6wGCbbQ/z-score-formula.jpg)

In [ ]:
min_train = X_train.min()
range_train = (X_train - min_train).max()
X_train = (X_train - min_train)/range_train

In [ ]:
min_test = X_test.min()
range_test = (X_test - min_test).max()
X_test = (X_test - min_test)/range_test

In [ ]:
X_test.isna().sum()

# Applying Machine Learning Model
Now comes the most interesting part applying machine learning model.In this step I will fit Random Forest Machine Learning Model.

## Random Forest: 
A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.RandomForestClassifie

The sub-sample size is always the same as the original input sample size but the samples are drawn with replacement if bootstrap=True (default).

In [ ]:
from sklearn.ensemble import RandomForestClassifier
Model=RandomForestClassifier(max_depth=2)
Model.fit(X_train,y_train)
y_pred=Model.predict(X_test)
print(classification_report(y_pred,y_test))
print(confusion_matrix(y_pred,y_test))
#Accuracy Score
print('accuracy is ',accuracy_score(y_pred,y_test))

# Feature Importance

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
rfc_model = RandomForestClassifier(random_state=0).fit(train_X, train_y)

Here is how to calculate and show importances with the eli5 library:

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(rfc_model, random_state=1).fit(val_X, val_y)
eli5.show_weights(perm, feature_names = val_X.columns.tolist(),top=13)


# Findings from the above graph

- As you move down the top of the graph, the importance of the feature decreases.
- The features that are shown in green indicate that they have a positive impact on our prediction
- The features that are shown in white indicate that they have no effect on our prediction
- The features shown in red indicate that they have a negative impact on our prediction

# Partial Dependence Plots
While feature importance shows what variables most affect predictions, partial dependence plots show how a feature affects predictions.

This is useful to answer questions like:

- Controlling for all other house features, what impact do longitude and latitude have on home prices? To restate this, how would similarly sized houses be priced in different areas?

- Are predicted health differences between two groups due to differences in their diets, or due to some other factor?

If you are familiar with linear or logistic regression models, partial dependence plots can be interepreted similarly to the coefficients in those models. Though, partial dependence plots on sophisticated models can capture more complex patterns than coefficients from simple models.

Like permutation importance, partial dependence plots are calculated after a model has been fit. The model is fit on real data that has not been artificially manipulated in any way.

[Credit](https://www.kaggle.com/learn/machine-learning-explainability)

For the sake of explanation, I use a Decision Tree which you can see below.

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
tree_model = DecisionTreeClassifier(random_state=0, max_depth=5, min_samples_split=5).fit(train_X, train_y)

In [ ]:
features = [c for c in X.columns if c not in ['Outcome']]

In [ ]:
from sklearn import tree
import graphviz
tree_graph = tree.export_graphviz(tree_model, out_file=None, feature_names=features)
display(graphviz.Source(tree_graph))

As guidance to read the tree:

- Leaves with children show their splitting criterion on the top
- The pair of values at the bottom show the count of True values and False values for the target respectively, of data points in that node of the tree.

# PDP Box

Here is the code to create the Partial Dependence Plot using the [PDPBox library](!https://pdpbox.readthedocs.io/en/latest/).

In [ ]:
from matplotlib import pyplot as plt
from pdpbox import pdp, get_dataset, info_plots

# Create the data that we will plot
pdp_goals = pdp.pdp_isolate(model=tree_model, dataset=val_X, model_features=features, feature='Glucose')

# plot it
pdp.pdp_plot(pdp_goals, 'Glucose')
plt.show()

A few items are worth pointing out as you interpret this plot

The y axis is interpreted as change in the prediction from what it would be predicted at the baseline or leftmost value.
- A blue shaded area indicates level of confidence
- From this particular graph, we see that increasing level of **Glucose** substantially increases the chances of having **"Diabetes"**. But Glucose level beyond 160  appear to have little impact on predictions.

Here is another example plot:

In [ ]:
feature_to_plot = 'BMI'
pdp_dist = pdp.pdp_isolate(model=tree_model, dataset=val_X, model_features=features, feature=feature_to_plot)

pdp.pdp_plot(pdp_dist, feature_to_plot)
plt.show()

In the above plot, it is evident that when the BMI value is between 28 and 42 it has very little impact on predictions. But when BMI value is beyond 42 it substantially increases the chance of diabetes. 

This graph seems too simple to represent reality. But that's because the model is so simple. You should be able to see from the decision tree above that this is representing exactly the model's structure.

You can easily compare the structure or implications of different models. Here is the same plot with a Random Forest model.

In [ ]:
# Build Random Forest model
rf_model = RandomForestClassifier(random_state=0).fit(train_X, train_y)

pdp_dist = pdp.pdp_isolate(model=rf_model, dataset=val_X, model_features=features, feature=feature_to_plot)

pdp.pdp_plot(pdp_dist, feature_to_plot)
plt.show()

In [ ]:
pdp_dist = pdp.pdp_isolate(model=rf_model, dataset=val_X, model_features=features, feature='Glucose')

pdp.pdp_plot(pdp_dist, 'Glucose')
plt.show()

# 2D Partial Dependence Plots
If you are curious about interactions between features, 2D partial dependence plots are also useful. An example may clarify what this.

We will again use the Decision Tree model for this graph. It will create an extremely simple plot, but you should be able to match what you see in the plot to the tree itself.

In [ ]:
# Similar to previous PDP plot except we use pdp_interact instead of pdp_isolate and pdp_interact_plot instead of pdp_isolate_plot
features_to_plot = ['Glucose', 'BMI']
inter1  =  pdp.pdp_interact(model=tree_model, dataset=val_X, model_features=features, features=features_to_plot)

pdp.pdp_interact_plot(pdp_interact_out=inter1, feature_names=features_to_plot,  plot_type='grid',
                                  plot_pdp=True)

plt.show()

# SHAP Values
You've seen (and used) techniques to extract general insights from a machine learning model. But what if you want to break down how the model works for an individual prediction?

SHAP Values (an acronym from SHapley Additive exPlanations) break down a prediction to show the impact of each feature. Where could you use this?

- A model says a bank shouldn't loan someone money, and the bank is legally required to explain the basis for each loan rejection
- A healthcare provider wants to identify what factors are driving each patient's risk of some disease so they can directly address those risk factors with targeted health interventions

You'll use SHAP Values to explain individual predictions.

### How They Work
SHAP values interpret the impact of having a certain value for a given feature in comparison to the prediction we'd make if that feature took some baseline value.

![](https://i.ibb.co/VQt0KKs/shap-diagram.png)

This section is based on this [course](!https://www.kaggle.com/dansbecker/shap-values) on kaggle.


In [ ]:
import shap  # package used to calculate Shap values

# Create object that can calculate shap values
explainer = shap.TreeExplainer(rfc_model)

# Calculate Shap values
shap_values = explainer.shap_values(X_train)

In [ ]:
shap.summary_plot(shap_values, X_train)

In [ ]:
shap.summary_plot(shap_values, X_test)

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], X_train.iloc[:,1:10])